# Setup 

In [ ]:
import google.generativeai as gai
import json, textwrap
import pandas as pd
from IPython.display import Markdown
gemini_key = json.load(open('keys.json'))['gemini']['api-key']
gai.configure(api_key=gemini_key)
MODEL = gai.GenerativeModel('gemini-pro')

# Define helper functions

In [ ]:
def reformat_response(resp):
    return "\n".join([line for line in resp.text.split('\n') if line.split()[0]==line.split()[0].upper() and line!="\n" and "`" not in line])

In [ ]:
def get_instructions(assignment, model):
    chat = model.start_chat(history=[])
    response = chat.send_message('Generate a sql query with each command on a different line such that it satisfies the following: '+assignment+ ". Do not explain and give no other text.")
    ret = []
    for query_line in reformat_response(response).split('\n'):
        ret+= [chat.send_message("Explain what the command "+query_line+"is doing in one sentence on one line without any formatting. Be specific about any tables, columns, or values that are used.").text.split("\n")[0]]
    return ret, chat

In [ ]:
def show_instructions(instructions):
    ret = f'The query should be made of {len(instructions)} commands that {"does" if len(instructions)==1 else "do"} the following:\n '
    for i, instruction in enumerate(instructions):
        ret+=f"{i+1}. {instruction}\n "
    return ret

# Run

In [ ]:
prompt = input("Enter the prompt: ")
query,_ = get_instructions(prompt, MODEL)
print(show_instructions(query))